# 2. Computer Vision

Librerías

In [1]:
import json
import re

Funbciones

In [2]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [7]:
def extract_info(data):
    registration_number = ""
    print_date = ""
    department = ""
    municipality = ""
    locality = ""
    status = ""
    
    # Diccionario para almacenar la información extraída
    info = {
        "registration_number": None,
        "print_date": None,
        "department": None,
        "municipality": None,
        "locality": None,
        "status": None
    }

    # Mapeo de nombres de meses en español a números
    month_mapping = {
        "Enero": "01", "Febrero": "02", "Marzo": "03", "Abril": "04",
        "Mayo": "05", "Junio": "06", "Julio": "07", "Agosto": "08",
        "Septiembre": "09", "Octubre": "10", "Noviembre": "11", "Diciembre": "12"
    }

    # Variables auxiliares para el estado del folio
    found_status_label = False

    # Iterar sobre los bloques de texto en el JSON
    for block in data.get('Blocks', []):
        if block.get('BlockType') == 'LINE':
            text = block.get('Text', '')

            # Buscar y extraer el número de matrícula
            if "Nro Matrícula" in text:
                registration_number = text.split(":")[-1].strip()
                info['registration_number'] = registration_number

            # Buscar y extraer la fecha de impresión y convertirla al formato YYYY-MM-DD
            if "Impreso el" in text:
                match = re.search(r'Impreso el (\d{1,2}) de (\w+) de (\d{4})', text)
                if match:
                    day = match.group(1)
                    month = match.group(2)
                    year = match.group(3)
                    month_number = month_mapping.get(month, "01")
                    print_date = f"{year}-{month_number.zfill(2)}-{day.zfill(2)}"
                    info['print_date'] = print_date

            # Buscar y extraer el departamento, municipio y vereda
            if "CIRCULO REGISTRAL" in text:
                depto_match = re.search(r'DEPTO:\s*([^ ]+)', text)
                municipio_match = re.search(r'MUNICIPIO:\s*([^ ]+)', text)
                vereda_match = re.search(r'VEREDA:\s*([^ ]+)', text)

                if depto_match:
                    department = depto_match.group(1).strip()
                    info['department'] = department

                if municipio_match:
                    municipality = municipio_match.group(1).strip()
                    info['municipality'] = municipality

                if vereda_match:
                    locality = vereda_match.group(1).strip()
                    info['locality'] = locality

            # Buscar y extraer el estado del folio
            if "ESTADO DEL FOLIO:" in text:
                found_status_label = True
            elif found_status_label:
                status = text.strip()
                info['status'] = status
                found_status_label = False

    return info

In [4]:
def main(file_path):
    data = load_json(file_path)
    info = extract_info(data)
    
    print("Número de Matrícula:", info['registration_number'])
    print("Fecha de Impresión:", info['print_date'])
    print("Departamento:", info['department'])
    print("Municipio:", info['municipality'])
    print("Localidad (Vereda):", info['locality'])
    print("Estado del Folio:", info['status'])

Proceso

In [8]:
if __name__ == "__main__":
    file_path = "G:/Mi unidad/Prueba Bancolombia/001-1007202-220301269555588250_pag1.json"
    main(file_path)

Número de Matrícula: 001-1007202
Fecha de Impresión: 2022-03-01
Departamento: ANTIOQUIA
Municipio: MEDELLIN
Localidad (Vereda): MEDELLIN
Estado del Folio: ACTIVO


In [10]:
if __name__ == "__main__":
    file_path = "G:/Mi unidad/Prueba Bancolombia/040-464031-220204640254303951_pag1.json"
    main(file_path)

Número de Matrícula: 040-464031
Fecha de Impresión: 2022-02-04
Departamento: ATLANTICO
Municipio: BARRANQUILLA
Localidad (Vereda): BARRANQUILLA
Estado del Folio: ACTIVO
